### Connecting to the European Soccer Database

In [188]:
import sqlite3
import pandas as pd

try:
    sqliteConnection = sqlite3.connect('database.sqlite')
    #cursor = sqliteConnection.cursor()
    print("Database created and Successfully Connected to SQLite")
except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)

Database created and Successfully Connected to SQLite


### Populating the 7 tables in the database to 7 Pandas Dataframes

In [189]:
country_query = "SELECT * from Country"
country_df = pd.read_sql_query(country_query, sqliteConnection)

league_query = "SELECT * from League"
league_df = pd.read_sql_query(league_query, sqliteConnection)

match_query = "SELECT * from Match"
match_df = pd.read_sql_query(match_query, sqliteConnection)

player_query = "SELECT * from Player"
player_df = pd.read_sql_query(player_query, sqliteConnection)

playerAttributes_query = "SELECT * from Player_Attributes"
playerAttributes_df = pd.read_sql_query(playerAttributes_query, sqliteConnection)

team_query = "SELECT * from Team"
team_df = pd.read_sql_query(team_query, sqliteConnection)

teamAttributes_query = "SELECT * from Team_Attributes"
teamAttributes_df = pd.read_sql_query(teamAttributes_query, sqliteConnection)

### Cleaning the Country Dataframe

In [190]:
del country_df['id']
country_df.insert(0, 'Country_id', range(1, len(country_df)+1))
country_df = country_df.rename(columns={"name": "Country_Name"})
country_df

,Country_id,Country_Name
0,1,Belgium
1,2,England
2,3,France
3,4,Germany
4,5,Italy
5,6,Netherlands
6,7,Poland
7,8,Portugal
8,9,Scotland
9,10,Spain


### Cleaning the League Dataframe

In [191]:
del league_df['id']
del league_df['country_id']
league_df.insert(0, 'League_id', range(1, len(league_df)+1))
league_df = league_df.rename(columns={"name": "League_Name"})
league_df

,League_id,League_Name
0,1,Belgium Jupiler League
1,2,England Premier League
2,3,France Ligue 1
3,4,Germany 1. Bundesliga
4,5,Italy Serie A
5,6,Netherlands Eredivisie
6,7,Poland Ekstraklasa
7,8,Portugal Liga ZON Sagres
8,9,Scotland Premier League
9,10,Spain LIGA BBVA


### Cleaning the Match Dataframe

In [192]:
match_df = match_df.rename(columns={"id": "Match_id"})

#Replacing the old country_id values with the Country_Name values for better readability
match_df['country_id'] = match_df['country_id'].replace(1,'Belgium')
match_df['country_id'] = match_df['country_id'].replace(1729,'England')
match_df['country_id'] = match_df['country_id'].replace(4769,'France')
match_df['country_id'] = match_df['country_id'].replace(7809,'Germany')
match_df['country_id'] = match_df['country_id'].replace(10257,'Italy')
match_df['country_id'] = match_df['country_id'].replace(13274,'Netherlands')
match_df['country_id'] = match_df['country_id'].replace(15722,'Poland')
match_df['country_id'] = match_df['country_id'].replace(17642,'Portugal')
match_df['country_id'] = match_df['country_id'].replace(19694,'Scotland')
match_df['country_id'] = match_df['country_id'].replace(21518,'Spain')
match_df['country_id'] = match_df['country_id'].replace(24558,'Switzerland')
match_df = match_df.rename(columns={"country_id": "Country_Name"})

#Replacing the old league_id values with the League_Name values for better readability
match_df['league_id'] = match_df['league_id'].replace(1,'Belgium Jupiler League')
match_df['league_id'] = match_df['league_id'].replace(1729,'England Premier League')
match_df['league_id'] = match_df['league_id'].replace(4769,'France Ligue 1')
match_df['league_id'] = match_df['league_id'].replace(7809,'Germany 1. Bundesliga')
match_df['league_id'] = match_df['league_id'].replace(10257,'Italy Serie A')
match_df['league_id'] = match_df['league_id'].replace(13274,'Netherlands Eredivisie')
match_df['league_id'] = match_df['league_id'].replace(15722,'Poland Ekstraklasa')
match_df['league_id'] = match_df['league_id'].replace(17642,'Portugal Liga ZON Sagres')
match_df['league_id'] = match_df['league_id'].replace(19694,'Scotland Premier League')
match_df['league_id'] = match_df['league_id'].replace(21518,'Spain LIGA BBVA')
match_df['league_id'] = match_df['league_id'].replace(24558,'Switzerland Super League')
match_df = match_df.rename(columns={"league_id": "League_Name"})
match_df['date'] = pd.to_datetime(match_df['date'], format='%Y/%m/%d %H:%M:%S')
match_df.head()
#Lesa na2es feeh shwayet hagat

,Match_id,Country_Name,League_Name,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-17,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-16,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-16,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-17,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,Belgium,Belgium Jupiler League,2008/2009,1,2008-08-16,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
